# 1b Centered Derivatives

We can approximate partial spatial derivatives with finite differences, and consider the function values $f_{i,jk}$ at discrete points $(i\Delta x,j\Delta y,k\Delta z)$ in three dimension to be variables whose time evolution is governed by ordinary differential equations (ODEs) in time. Using for example a 2nd order finite difference to approximate the spatial derivatives, and looking at a case with constant velocity ${\bf v}$ and a profile $f(r,t)$ that is time independent in the moving coordinate frame ($f({\bf r}_0-{\bf v}t,t) = f({\bf r}_0,0)$), we will need to compute spatial derivatives, with formulae such as

\begin{equation}\tag{1}
\left(\frac{\partial f}{\partial x}\right)_{i,j,k} = \frac{1}{2\Delta x} \left(f_{i+1,j,k}-f_{i-1,j,k}\right)
\end{equation}

The finite difference approximation for the derivative can be derived by considering Taylor series of the function (and therefore assumes that the function is well behaved). We can write:
\begin{align}\tag{2}
f_{i+1,j,k} & = f_{i,j,k} + \left(\frac{\partial f}{\partial x}\right)_{i,j,k} \Delta x + {1 \over 2} \left(\frac{\partial^2 f}{\partial x^2}\right)_{i,j,k} \Delta x^2 + \mathcal{O}(\Delta x^3) \\
f_{i-1,j,k} & = f_{i,j,k} - \left(\frac{\partial f}{\partial x}\right)_{i,j,k} \Delta x + {1 \over 2} \left(\frac{\partial^2 f}{\partial x^2}\right)_{i,j,k} \Delta x^2 + \mathcal{O}(\Delta x^3)
\end{align}
Subtracting the two to remove $f_{i,j,k}$ and rearranging terms we see that in the limit where $\Delta x$ becomes small the error, or difference, between the finite difference equation and the correct value for the derivative should scale like $\Delta x^2$. This is called a second order approximation. Notice that this is an asymptotic scaling; a priori we do not know how large the constant of this second error is, and it entirely depends on the smoothness of the function involved.

To use Python to play around with this, we need a function that returns the spatial derivative in each of the three directions.  To avoid having to specify boundary conditions we assumme that the domain is periodic.  Then we can use the `numpy` function `roll` to quickly and efficiently compute the finite differences.

While we could put import and the derivative function together, it is good practice to have a seperate cell at the top that imports the neccessary modules. Further down, we will need to do plotting. Therefore, we also import the `matplotlib.pyplot` module. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def deriv(f,ds,axis=0):
    """ 
    Compute derivative in the axis-direction, given f[i,j,k] 
    and ds = mesh size in the axis-direction
    """
    return (np.roll(f,-1,axis)-np.roll(f,+1,axis))/(2.0*ds)

`numpy.roll` (or `np.roll`, because we import `numpy` as `np`) does a "circular shift" (shifting with periodic wrapping). To see documentation for it, do

In [ ]:
help(np.roll)

To try the partial derivative on a sin(x) function we create a 3D array with a `sin(x)` function along the x-axis.

Notice that we put a `;` on the last line, this will prevent Jupyter in automatically trying to print the return value from the last function, which in this case will just be some info about what and where things have been plotted.

If you get annoyed by an output window with scroll bars then right-click anywhere on the notebook window and select _Disable scrolling for outputs_

In [ ]:
def sinx_3d(m):
    f=np.zeros(m)
    x=np.zeros(m[0])
    ds=2.0*np.pi/np.array(m)
    for i in range(m[0]):
        x[i]=i*ds[0]
        f[i,:,:]=np.sin(x[i])
    return f,x,ds
n=128
m=(n,n,n)
f,x,ds=sinx_3d(m)
plt.plot(x,f[:,0,0])
plt.xlabel('x')
plt.ylabel('f(x)');

Check the derivative, which should be $cos(x)$

In [ ]:
dfdx=deriv(f,ds[0],0)
plt.plot(x,dfdx[:,0,0],label='numerical derivative')
plt.xlabel('x')
plt.ylabel('$\partial f/\partial x$')
plt.plot(x,np.cos(x),'+',label='analytical derivative')
plt.legend();

The `deriv` function also works for 1-D arrays:

In [ ]:
f=np.sin(x)
dx=ds[0]
dfdx=deriv(f,dx)
plt.plot(x,f);

Let's check the size of the error:

In [ ]:
dfdx=deriv(f,dx)
plt.plot(x,dfdx,label='derivative')
plt.xlabel('x')
plt.ylabel('$\partial f/\partial x$')
plt.plot(x,1e3*(dfdx-np.cos(x)),label='error x1000')
plt.legend();

Evidently, the error in the finite difference derivative is a very small amount (less than one part in a thousand), with the given resolution (128 points in 2$\pi$). We can also see that the error is largest where the curvature of the derivative is largest.   

With lower resolution, the error grows, of course.  How do you think it scales with the resolution?  You can check your answer by modifying the code above (or copying the relevant cells below here):

In [ ]:
for n in (8,16,32,64,128):
    dx=2.*np.pi/n
    x=dx*np.arange(n)
    f=np.sin(x)
    d=max(abs(np.cos(x)-deriv(f,dx)))
    print('n: {:3d}  error: {:.6f}'.format(n,d))

We can compute the error for 2nd order derivatives, and make a loglog plot:

In [ ]:
nn=[]; d2=[]
for n in (8,16,32,64,128):
    dx=2.*np.pi/n
    x=dx*np.arange(n)
    f=np.sin(x)
    nn.append(n)
    d=max(abs(np.cos(x)-deriv(f,dx)))
    d2.append(d)
plt.loglog(nn,d2,'o',linestyle=':')
plt.xlabel('n'); plt.ylabel('error');

Watch out! In the above example I have defined the error as the maximum of the absolute difference between the analytical derivative and the finite difference approximation. How to define the error is somewhat arbitrary; one could also define it as the typical difference or even the RMS (second order) norm. The important part is to specify what has been done, and sometimes also understand why we get the results we get. E.g. a certain method could give excellent results except for at specific points. A maximum norm, as used above, is sensitive to outlier points.

## Task: Compare with 4th order centered derivative

The formula for a 4th order centered derivative is:

\begin{equation}\tag{2}
\left(\frac{\partial f}{\partial x}\right)_{i,j,k} = 
\frac{1}{12 \Delta x} 
\left[
8(f_{i+1,j,k}-f_{i-1,j,k})
-(f_{i+2,j,k}-f_{i-2,j,k})
\right]
\end{equation}

This can be shown be expanding in Taylor series for the four different points and see how terms cancel. Notice that while the error term scales as $(\Delta x)^4$, the prefactor is the fifth derivative of the function. Therefore approximating the derivative of a function with sudden changes -- e.g. a step function -- may result in large errors.

__Task:__ Compute the corresponding errors for the 4th order derivatives, and plot them as filled squares (Python symbol `'s'`), together with the 2nd order errors as filled circles (symbol `'o'`).

__Bonus task:__ Try to increase the number of points from 128 to 1e3, 1e4, 1e5, 1e6, and 1e7 for 2nd and 4th order. Do you understand what goes on with the error?

### __Absalon turn in:__

   1. Replace the ellipses (...) in the following sentence with the 4th order result, and add the code to the notebook below this cell.
   2. Upload the notebook as a pdf

_So, while the error at `n=128` is less than a part in a thousand for 2nd order, it's less than ... with 4th order! The error scales as ... for 4th order_